Tesla Stock and Revenue Data Visualization Project

Project Objective
The goal of this project is to extract Tesla Inc. (TSLA) historical stock and revenue data using two different methods and then visualize the data using interactive plots.

We will:

Use the yfinance library to extract Tesla’s historical stock data.

Use web scraping techniques with BeautifulSoup to extract Tesla’s historical revenue data from a webpage.

Combine both datasets into a single graph to compare Tesla’s share price and revenue over time.

In [ ]:
# Tools and Libraries Used
# Before we begin, let's install the necessary libraries (if not already installed):
!pip install yfinance
!pip install bs4
!pip install nbformat
!pip install --upgrade plotly

In [ ]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
#Set default renderer for Plotly and suppress warnings
import plotly.io as pio
pio.renderers.default = "iframe"

In [ ]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
#We'll define a helper function to plot Tesla’s share price and revenue on a two-row subplot.
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

Technique 1 – Extract Stock Data Using yfinance

We will use the yfinance.Ticker() object to get Tesla’s historical stock data.

In [ ]:
# Create a Ticker object for Tesla
tesla = yf.Ticker('TSLA')

# Use the 'max' period to get the longest available history
tesla_data = tesla.history(period="max")

# Reset the index to make 'Date' a column instead of index
tesla_data.reset_index(inplace=True)

# Preview the data
tesla_data.head()

Technique 2 – Extract Revenue Data via Web Scraping

Now we will scrape Tesla’s historical revenue data from an HTML page using BeautifulSoup.

In [ ]:
# URL containing Tesla revenue data
url = ' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm'
# Get the HTML content
html_data = requests.get(url).text

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(html_data,'html.parser')

#Create an empty DataFrame and extract revenue table:
tesla_revenue = pd.DataFrame(columns = ['Date','Revenue'])

# First we isolate the body of the table which contains all the information
# Then we loop through each row and find all the column values for each row
for row in soup.find_all("tbody")[1].find_all('tr'):
    col = row.find_all("td")
    date = col[0].text
    revenue = col[1].text
    
    # Finally we append the data of each row to the table
    tesla_revenue = pd.concat([tesla_revenue,pd.DataFrame({"Date":[date], "Revenue":[revenue]})], ignore_index=True)    

In [ ]:
#Clean and convert the revenue column:

# Strip whitespace and remove commas and dollar signs
tesla_revenue["Revenue"] = tesla_revenue["Revenue"].str.strip()
tesla_revenue["Revenue"] = tesla_revenue["Revenue"].str.replace(r'[\$,]', '', regex=True)

#Remove an null or empty strings in the Revenue column.
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]
tesla_revenue['Revenue'] = pd.to_numeric(tesla_revenue['Revenue'])

#Last 5 rows of the dataframe
tesla_revenue.tail()

Final Step – Plot Tesla Stock Price vs Revenue

Now we will use the make_graph() function to plot the two datasets:

In [ ]:
make_graph(tesla_data, tesla_revenue, 'Tesla')

Conclusion

In this mini project, we:

Extracted Tesla’s stock price history using the yfinance API.

Scraped Tesla’s revenue data from an online source using BeautifulSoup.

Cleaned and prepared the data using pandas.

Visualized the stock price and revenue trends using Plotly.

This provides a helpful view of how Tesla’s financial performance (revenue) aligns with its market performance (share price) over time.